## Test all the things!
* Write/update transactions to get confirmation of success/failure
* Custom indexes to make querying by properties possible

In [33]:
from google.cloud import datastore
from datetime import datetime

PROJECT = 'reliable-realm-222318'

In [34]:
client = datastore.Client(project=PROJECT)

### Transactions

In [35]:
with client.transaction() as xact:
    key = client.key('Task')
    item = datastore.Entity(key=key)
    item['description'] = 'return a value maybe?'
    client.put(item)
    mutations = xact.mutations

In [45]:
mut = mutations[0]

In [53]:
from google.protobuf.json_format import MessageToDict

In [56]:
mut_dict = MessageToDict(mut)

In [60]:
mut_dict['insert']

{'key': {'partitionId': {'projectId': 'reliable-realm-222318'},
  'path': [{'kind': 'Task'}]},
 'properties': {'description': {'stringValue': 'return a value maybe?'}}}